# AgriMarket

## 농수산물


- Data Source : [농식품 빅데이터 거래소](https://kadx.co.kr/product/detail/94534d90-595d-11eb-acdd-9560621d9652)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- 일반농산물 도매 69품목 116품종, 소매 90품목 143품종 및 친환경농산물 38품목 44종 대상 가격 자료


## 소비자 물가지수 


- Data Source : [국가통계포털](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1J20112&conn_path=I2)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- 농산물 중 필수재라고 판단되는 곡물,채소 포함

- 사치재인 과실, 기타 농산물 제외

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'

root_path = "../"
data_path = f"{root_path}data"

col_detail={
'prce_reg_ymd' : '가격등록일자',
'mrkt_esnt_no' : '시장고유번호',
'mrkt_code' : '시장코드',
'mrkt_nm' : '시장명',
'ct_code' : '시도코드',
'ctnp_nm' : '시도명',
'sggu_code' : '시군구코드',
'sggu_nm' : '시군구명',
'pdlt_code' : '품목코드',
'pdlt_nm' : '품목명',
'spcs_code' : '품종코드',
'spcs_nm' : '품종명',
'wsrt_exmn_se_code' : '도소매조사구분코드',
'exmn_se_nm' : '조사구분명',
'bulk_grad_code' : '산물등급코드',
'bulk_grad_nm' : '산물등급명',
'pdlt_prce' : '품목가격',
'mtc_smt_unit_mg' : '산지출하단위크기',
'mtc_smt_unit_nm' : '산지출하단위명',
'whsl_smt_unit_mg' : '도매출하단위크기',
'whsl_smt_unit_nm' : '도매출하단위명',
'rtsl_smt_unit_mg' : '소매출하단위크기',
'rtsl_smt_unit_nm' : '소매출하단위명',
'evfd_fmpd_smt_unit_mg' : '친환경농산물출하단위크기',
'evfd_fmpd_smt_unit_nm' : '친환경농산물출하단위명',
'dcnt_prce_yn' : '할인가격여부',
'etl_ldg_dt' : 'ETL적재일시'
}

# 파일 호출, 변수명 및 불러오기 경로는 자신에게 맞게 변경해주시면 됩니다.

In [10]:
import os
files_path = f'{data_path}/TB_TAT_KAMIS_PRC_MK_TB/'
files = os.listdir(files_path)
# files.remove('.DS_Store')
print(files)

['TB_TAT_KAMIS_PRC_MK_TB-1996.csv', 'TB_TAT_KAMIS_PRC_MK_TB-1997.csv', 'TB_TAT_KAMIS_PRC_MK_TB-1998.csv', 'TB_TAT_KAMIS_PRC_MK_TB-1999.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2000.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2001.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2002.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2003.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2004.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2005.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2006.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2007.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2008.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2009.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2010.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2011.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2012.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2013.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2014.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2015.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2016.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2017.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2018.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2019.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2020.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2021.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2022.csv']


In [11]:
df = pd.DataFrame()
for file in files:
    df_sub = pd.read_csv(f'{files_path}/{file}', encoding='cp949', low_memory=False)
    df_sub = df_sub[['PRCE_REG_YMD', 'CTNP_NM', 'PDLT_NM', 'EXMN_SE_NM', 'BULK_GRAD_NM', 'PDLT_PRCE']]
    df_sub = df_sub[~df_sub['PDLT_NM'].isna()]
    df_sub.columns=['YMD', 'Region', 'Product', 'WR', 'Grade', 'Price']
    df = pd.concat([df, df_sub])

# 1. 년·월·일 에서 연·월 로 변경
# 2. Y(년), M(월) 컬럼 생성

In [12]:
df['YMD'] = df['YMD'].astype(str)

df['Y'] = df['YMD'].map(lambda x:x[:4]).astype('int16')
df['M'] = df['YMD'].map(lambda x:x[4:6]).astype('int8')
df['D'] = df['YMD'].map(lambda x:x[6:]).astype('int8')
df['YM'] = df['YMD'].map(lambda x:x[:6]).astype('int32')

In [13]:
# 20년 데이터만 있는 품목 제외

df = df[~df['Product'].isin(['즉석밥', '두부', '김치', '고추장', '된장', '간장', '맛김(조미김)', '콩나물'])]

In [14]:
df.loc[df["Product"]=="피마늘", "Product"] = "마늘"

In [15]:
# Wholesale : 도매
df_w = df[df['WR']=='도매']

In [16]:
# Retail : 소매
df_r = df[df['WR']=='소매']

In [17]:
product_w = df_w['Product'].unique()
product_w = product_w.tolist()

In [18]:
product_r = df_r['Product'].unique()
product_r = product_r.tolist()

In [19]:
df

,YMD,Region,Product,WR,Grade,Price,Y,M,D,YM
0,19960103,부산,쌀,도매,상품,34000.0,1996,1,3,199601
1,19960103,대구,쌀,도매,상품,33500.0,1996,1,3,199601
2,19960103,인천,쌀,도매,상품,33500.0,1996,1,3,199601
3,19960103,광주,쌀,도매,상품,33250.0,1996,1,3,199601
4,19960103,대전,쌀,도매,상품,33250.0,1996,1,3,199601
...,...,...,...,...,...,...,...,...,...,...
1111622,20221215,충북,새우젓,소매,상품,25000.0,2022,12,15,202212
1111623,20221215,충북,멸치액젓,소매,상품,5000.0,2022,12,15,202212
1111624,20221215,충북,굵은소금,소매,상품,12980.0,2022,12,15,202212
1111625,20221215,충북,전복,소매,중품,19670.0,2022,12,15,202212


# 물가지수 파일과 겹치는 품목 확인

In [20]:
df_1 = pd.read_csv(f'{data_path}/소비자물가지수_품목별_연도별_월별_filtered.csv')

In [21]:
df_1['품목별'].unique()

array(['총지수', '농축수산물', '\u3000\u3000\u3000(농산물)곡물',
       '\u3000\u3000\u3000\u3000\u3000\u3000쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000현미',
       '\u3000\u3000\u3000\u3000\u3000\u3000찹쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000보리쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000콩',
       '\u3000\u3000\u3000\u3000\u3000\u3000땅콩',
       '\u3000\u3000\u3000\u3000\u3000\u3000혼식곡',
       '\u3000\u3000\u3000(농산물)채소',
       '\u3000\u3000\u3000\u3000\u3000\u3000배추',
       '\u3000\u3000\u3000\u3000\u3000\u3000상추',
       '\u3000\u3000\u3000\u3000\u3000\u3000시금치',
       '\u3000\u3000\u3000\u3000\u3000\u3000양배추',
       '\u3000\u3000\u3000\u3000\u3000\u3000미나리',
       '\u3000\u3000\u3000\u3000\u3000\u3000깻잎',
       '\u3000\u3000\u3000\u3000\u3000\u3000부추',
       '\u3000\u3000\u3000\u3000\u3000\u3000무',
       '\u3000\u3000\u3000\u3000\u3000\u3000열무',
       '\u3000\u3000\u3000\u3000\u3000\u3000당근',
       '\u3000\u3000\u3000\u3000\u3000\u3000감자',
       '\u3000\u3000\u3000\u

In [22]:
# replace로 \u3000 값이 사라지지 않는다.. 이유는 모르겠음
df_1['품목별'] = df_1['품목별'].replace('\u3000', '')

In [23]:
df_1['품목별'] = df_1['품목별'].map(lambda x:x.strip(('\u3000')))

In [24]:
df['Product'].unique()

array(['쌀', '찹쌀', '콩', '팥', '녹두', '고구마', '감자', '배추', '양배추', '시금치', '상추',
       '오이', '호박', '무', '당근', '건고추', '풋고추', '붉은고추', '마늘', '양파', '파', '생강',
       '미나리', '참깨', '들깨', '땅콩', '사과', '배', '감귤', '단감', '바나나', '참다래',
       '파인애플', '고등어', '갈치', '명태', '물오징어', '건멸치', '북어', '건오징어', '김', '건미역',
       '굴', '깐마늘(국산)', '돼지고기', '닭고기', '계란', '딸기', '토마토', '참외', '수박',
       '복숭아', '포도', '깻잎', '오렌지', '방울토마토', '우유', '피망', '느타리버섯', '메밀',
       '깐마늘(수입)', '팽이버섯', '수입조기', '쇠고기', '파프리카', '멜론', '새송이버섯', '얼갈이배추',
       '열무', '건포도', '건블루베리', '레몬', '체리', '아몬드', '자몽', '호두', '고춧가루',
       '굵은소금', '멸치액젓', '새우젓', '망고', '갓', '가지', '부추', '새우', '전복', '꽁치',
       '브로콜리', '청경채', '케일', '양송이버섯', '표고버섯', '귀리', '보리', '수수', '율무',
       '양상추', '혼합곡', '기장', '연근', '우엉', '아보카도', '블루베리', '조기', '절임배추'],
      dtype=object)

# 농수산물 데이터와 주요경제지표 합치기

In [25]:
file = f"{data_path}/주요경제지표_20052021.xlsx"
df_mei = pd.read_excel(file, engine="openpyxl")
print(df_mei.shape)
display(df_mei.head(df_mei.shape[0]))

(17, 11)


c:\Users\sdy59\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,연도,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,2021,110.64,108.050,88302.7,-154.3,463118.360,644400.370,615093.450,1144.42,3.7,0.61
1,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.730,467548.715,1180.05,3.9,0.70
2,2019,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.80,3.8,1.59
3,2018,104.46,104.450,76408.5,316.7,403694.264,604859.657,535202.428,1118.10,3.8,1.52
4,2017,102.53,102.930,78460.0,-26.8,389266.683,573694.421,478478.296,1071.40,3.7,1.26
5,2016,99.11,100.970,98677.4,-46.2,371101.605,495425.934,406192.887,1208.50,3.7,1.34
6,2015,100.95,100.000,105939.6,-60.2,367961.865,526756.498,436498.973,1172.00,3.6,1.65
7,2014,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.20,3.5,2.34
8,2013,105.73,98.048,81148.2,-27.0,346459.569,559632.400,515585.500,1055.30,3.1,2.59
9,2012,107.45,96.789,50835.0,-41.7,326968.393,547869.800,519584.500,1071.10,3.2,3.08


In [26]:
df_mei = df_mei[df_mei["연도"] <= 2020]

In [27]:
df_mei = df_mei.rename(columns={"연도" : "Y"})
df_mei

,Y,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
1,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.730,467548.715,1180.05,3.9,0.70
2,2019,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.80,3.8,1.59
3,2018,104.46,104.450,76408.5,316.7,403694.264,604859.657,535202.428,1118.10,3.8,1.52
4,2017,102.53,102.930,78460.0,-26.8,389266.683,573694.421,478478.296,1071.40,3.7,1.26
5,2016,99.11,100.970,98677.4,-46.2,371101.605,495425.934,406192.887,1208.50,3.7,1.34
6,2015,100.95,100.000,105939.6,-60.2,367961.865,526756.498,436498.973,1172.00,3.6,1.65
7,2014,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.20,3.5,2.34
8,2013,105.73,98.048,81148.2,-27.0,346459.569,559632.400,515585.500,1055.30,3.1,2.59
9,2012,107.45,96.789,50835.0,-41.7,326968.393,547869.800,519584.500,1071.10,3.2,3.08
10,2011,106.71,94.717,18655.8,-112.0,306402.488,555213.700,524413.100,1153.30,3.4,3.09


In [28]:
df_market = pd.merge(df, df_mei)
df_market

,YMD,Region,Product,WR,Grade,Price,Y,M,D,YM,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,20050103,서울,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
1,20050103,부산,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
2,20050103,대구,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
3,20050103,광주,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
4,20050103,대전,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14237905,20201231,서울,청경채,07,무농약,4470.0,2020,12,31,202012,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237906,20201231,서울,팽이버섯,07,무농약,930.0,2020,12,31,202012,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237907,20201231,서울,상추,07,유기농,4940.0,2020,12,31,202012,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237908,20201231,서울,토마토,07,무농약,16800.0,2020,12,31,202012,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70


In [29]:
crop_list = [
    "호박",
    "고구마",
    "양파",
    "무",
    "배",
    "배추",
    "마늘",
    "양배추",
    "시금치",
    "콩",
    "땅콩",
    "풋고추",
    "오이",
    "깻잎",
    "생강",
    "사과",
    "포도",
    "당근",
    "복숭아",
    "미나리",
    "감자",
    "열무",
    "쌀",
]

len(crop_list)

23

In [30]:
df_market = df_market[df_market["Product"].isin(crop_list)]
df_market

,YMD,Region,Product,WR,Grade,Price,Y,M,D,YM,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,20050103,서울,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
1,20050103,부산,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
2,20050103,대구,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
3,20050103,광주,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
4,20050103,대전,쌀,도매,상품,40000.0,2005,1,3,200501,86.88,78.444,12654.8,-0.6,210390.703,284418.70,261238.300,1013.00,3.7,3.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14237894,20201231,서울,풋고추,07,무농약,4850.0,2020,12,31,202012,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237895,20201231,서울,풋고추,07,유기농,4550.0,2020,12,31,202012,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237896,20201231,서울,풋고추,07,유기농,3710.0,2020,12,31,202012,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70
14237898,20201231,서울,양파,07,무농약,5950.0,2020,12,31,202012,104.01,105.420,75275.7,-386.3,443098.120,512788.73,467548.715,1180.05,3.9,0.70


In [31]:
df_market["Product"].nunique()

23

In [32]:
set(crop_list) - set(df_market["Product"])

set()

In [63]:
file = f"{data_path}/생산자물가지수_품목별_연도별_월별.xlsx"
df2 = pd.read_excel(file, engine="openpyxl")
df2 = df2.rename(columns={'계정코드별':'Product'})
df2["Product"] = df2["Product"].map(lambda x : x.split()[0])
df2 = df2.replace("-", np.NaN)
# df2 = df2.dropna(axis=0)
# df2 = df2[df2["Product"].isin(crop_list)]
# for i in df2["Product"]:
#     df2["Product"] = df2["Product"].replace(i,f"생산자물가지수_{i}")

for i in df2.columns:
    df2.columns = df2.columns.map(lambda x: x.replace(".",""))
    
# df2.stack()
df2 = pd.DataFrame(df2).T
df2 = df2.rename(columns=df2.iloc[0])
df2 = df2.drop(index='Product')
df2 = pd.DataFrame(df2.stack())
df2.columns = ['item_PPI']
df2

item_PPI
200501 쌀      101.54
       보리쌀     58.37
       콩      113.85
       감자      71.15
       고구마     53.12
...              ...
202211 한약재료    87.87
       잎담배    108.77
       참깨      137.1
       들깨     150.42
       생우유    107.75

[7750 rows x 1 columns]

In [57]:
df3 = pd.read_excel(f"{data_path}/소비자물가지수_품목별_연도별_월별.xlsx")
df3 = df3.drop(columns=['시도별'], axis=0)
df3 = df3.replace("-", np.NaN)
# df3 = df3.rename(columns={'품목별':'YM'}).set_index("YM", drop=True)
# df3 = df3.dropna(axis="rows")ㅋㅋㅋㅋㅋ짜증나게하지마세ㅕㅇ 
# # df3 = df3.drop(index=[0,1,2,10,38])
# # df3["YM"] = df3["YM"].map(lambda x: x.strip())
# # df3 = df3[df3["YM"].isin(crop_list)]
# # for i in df3["YM"]:
# #     df3["YM"] = df3["YM"].replace(i,f"소비자물가지수_{i}")

for i in df3.columns:
    df3.columns = df3.columns.map(lambda x: x.replace(".",""))
        

df3 = pd.DataFrame(df3).T
df3 = df3.rename(columns=df3.iloc[0])
df3 = df3.drop(index='품목별')
df3

,총지수,농축수산물,(농산물)곡물,쌀,현미,찹쌀,보리쌀,콩,땅콩,혼식곡,...,망고,체리,아보카도,파인애플,아몬드,(농산물)기타농산물,고춧가루,참깨,인삼,화초
200501,73.594,62.157,76.091,75.464,NaN,78.245,61.778,68.828,53.74,NaN,...,NaN,NaN,NaN,NaN,NaN,46.737,42.814,64.624,77.931,53.816
200502,73.892,64.05,76.015,75.539,NaN,76.985,61.839,68.56,53.166,NaN,...,NaN,NaN,NaN,NaN,NaN,47.521,42.48,65.48,79.06,64.15
200503,74.338,64.302,75.866,75.391,NaN,76.12,61.778,68.022,52.225,NaN,...,NaN,NaN,NaN,NaN,NaN,47.06,42.356,65.414,79.785,55.327
200504,74.413,64.681,75.641,75.093,NaN,77.694,61.839,68.09,52.172,NaN,...,NaN,NaN,NaN,NaN,NaN,46.829,42.605,65.545,80.349,47.576
200505,74.338,63.482,76.015,75.242,NaN,81.236,61.901,67.755,53.061,NaN,...,NaN,NaN,NaN,NaN,NaN,46.921,42.605,66.269,80.833,48.697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202207,108.74,114.4,97.32,95.23,102.33,97.55,106.59,103.04,107.11,103.82,...,119.94,139.61,95.43,118.23,98.91,105.13,103.9,108.79,96.6,110.21
202208,108.62,116.74,95.47,92.8,101.34,95.74,108.89,103.01,106.28,103.72,...,130.91,123.58,82.4,120.91,98.73,106.66,104.23,110.32,97.0,116.15
202209,108.93,118.99,93.93,90.34,101.72,97.04,109.55,102.52,106.4,104.2,...,136.55,118.73,85.37,131.93,98.72,111.83,108.11,111.29,97.59,130.54
202210,109.21,114.68,94.4,91.08,101.01,96.87,110.14,102.75,106.03,104.08,...,136.17,114.09,92.72,132.23,99.05,112.49,110.43,114.63,96.38,124.62


In [35]:
# 이게 우신님꺼
df_eco = pd.concat([df2, df3], axis = 0).set_index("YM").T
df_eco

KeyError: "None of ['YM'] are in the columns"

In [ ]:
df_eco["YM"]

KeyError: 'YM'

In [ ]:
pd.merge(df_market, df_eco, on="YM")

KeyError: 'YM'